<a href="https://colab.research.google.com/github/johyunkang/python-ml-guide/blob/main/python_ml_perfect_guide_08_TextAnal_05SentimentAnal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 05 감성분석

- 감성 분석은 머신러닝 관점에서 지도학습, 비지도학습으로 나눌 수 있음
    - 지도학습은 학습 데이터와 타깃 레이블 값을 기반으로 감성 분석 학습을 수행한 뒤 이를 기반으로 다른 데이터의 감성 분석을 예측하는 방법으로 일반적인 텍스트 기반의 분류와 거의 동일함
    - 비지도학습은 'Lexicon'이라는 일종의 감성 어휘 사전을 이용. Lexicon은 감성 분석을 위한 용어와 문맥에 대한 다양한 정보를 가지고 있으며, 이를 이용해 문서의 긍정적, 부정적 감성 여부를 판단함

#### 지도학습 기반 감성 분석 실습 - IMDB 영화평

In [20]:
import pandas as pd

review_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/text-anal/labeledTrainData.tsv.zip',
                        header=0, sep="\t", quoting=3)
review_df.head()



,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


- 데이터 의미
    - id : 각 데이터 id
    - sentiment : 영화평의 Sentiment 결과 값(Target Label). 1은 긍정적 평가, 0은 부정적 평가
    - review : 영화평의 텍스트

In [21]:
print(review_df['review'][0])

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

In [22]:
import re # 정규표현식 모듈

# <br> html 태그는 replace 함수로 공백 변환
review_df['review'] = review_df['review'].str.replace('<br />', ' ')

# 파이썬의 정규 표현식 모듈인 re를 이용해 영어 문자열이 아닌 문자는 모두 공백으로 변환
review_df['review'] = review_df['review'].apply(lambda x : re.sub("[^a-zA-Z]", " ", x))

- 결정 값 클래스인 sentiment 컬럼 별도 추출해 결정 값 데이터 세트 생성

In [23]:
from sklearn.model_selection import train_test_split

class_df = review_df['sentiment']
feature_df = review_df.drop(['id', 'sentiment'], axis=1, inplace=False)

x_train, x_test, y_train, y_test = train_test_split(feature_df, class_df, test_size=0.3, random_state=156)
x_train.shape, x_test.shape

((17500, 1), (7500, 1))

- Pipeline 을 이용해 Count 벡터화 적용해 예측 성능을 측정

In [24]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

# 스톱 워드는 english, filtering, ngram은 (1, 2)로 설정해 CountVectorization 수행
# 선형회귀의 C는 10으로 설정
pipeline = Pipeline([
    ('cnt_vect', CountVectorizer(stop_words='english', ngram_range=(1, 2))),
    ('lr_clf', LogisticRegression(C=10))
])

# pipeline 객체를 이용해 fit(), predict()로 학습/예측 수행
# predict_proba()는 roc_auc 때문에 수행
pipeline.fit(x_train['review'], y_train)
pred = pipeline.predict(x_test['review'])
pred_probs = pipeline.predict_proba(x_test['review'])[:, 1]

print('예측 정확도는 {0:.4f}, ROC-AUC는 {1:.4f}'.format(accuracy_score(y_test, pred),
                                                 roc_auc_score(y_test, pred)))


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


예측 정확도는 0.8860, ROC-AUC는 0.8859


> LogisticRegression의 C는 하이퍼 파라미터. 높은 C를 설정할 수록, 낮은 강도의 제약조건이 설정되고, 낮은 C를 설정할 수록, 높은 강도의 제약조건이 설정됨

- 이번에는 TF-IDF 벡터화를 이용해 성능 측정

In [ ]:
# stop words는 english , filtering, ngram은 (1, 2)로 설정해 TF-IDF 벡터화 수행
# LogisticRegression 의 하이퍼 파라미터 C는 10으로 설정
pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english', ngram_range=(1, 2))),
    ('lr_clf', LogisticRegression(C=10))
])

pipeline.fit(x_train['review'], y_train)
pred = pipeline.predict(x_test['review'])
pred_proba = pipeline.predict_proba(x_test['review'])[:, 1]

print('예측 정확도는 {0:.4f}, ROC-AUC는 {1:.4f}'.format(accuracy_score(y_test, pred),
                                                 roc_auc_score(y_test, pred)))

- Count 벡터화의 예측 정화도는 0.8860, ROC-AUC는 0.8859
- TF-IDF 벡터화의 예측 정확도는 0.8936, ROC-AUC는 0.8934
- TF-IDF 기반의 피처 벡터화의 예측 성능이 조금 더 나아졌음

#### 비지도학습 기반 감성 분석 소개

- 비지도 감성 분석은 Lexicon을 기반으로 하는 것입니다.
- Lexicon은 일반적으로 어휘집을 의미하지만 여기서는 감성만을 분석하기 위해 지원하는 감성 어휘 사전임.
- 감성사전은 긍정(Positive) 또는 부정(Negative) 수치를 가지고 있으며 이를 감성지수(Polarity score)라고 함
- 감성 지수는 단어의 위치나 주변 단어, 문맥, POS(Part of Speech) 등을 참고해 결정됨

#### SentiWordNet을 이용한 감성 분석

In [ ]:
import nltk
nltk.download('all')

- 'present' 단어에 대한 Synset을 추출해 보겠음
- WordNet의 synsets()는 파라미터로 지정된 단어에 대해 WordNet에 등재된 모든 Synset 객체를 반환함

In [ ]:
from nltk.corpus import wordnet as wn

term = 'present'

# 'present'라는 단어로 wordnet의 synsets 생성
synsets = wn.synsets(term)
print('synsets() 반환 type:', type(synsets))
print('synsets() 반환 값 개수:', len(synsets))
print('synsets() 반환 값:', synsets)

- 위의 Synset 객체를 가지는 리스트는 총 18개임
- Synset('present.n.01')와 같이 Synset 객체의 파라미터 'present.n.01'은 POS 태그를 나타냄
- 'present.n.01'에서 present는 의미, n은 명사 품사, 01은 present가 명사로서 가지는 의미가 여러 가지 있어서 이를 구분하는 인덱스

In [ ]:
for synset in synsets :
    print('#### Synset name:', synset.name(), '####')
    print('POS:', synset.lexname())
    print('Definition:', synset.definition())
    print('Lemmas:', synset.lemma_names())
    print('\n\n')

- WordNet은 어떤 어휘와 다른 어휘 간의 관계를 유사도로 나타냄
- synset 객체는 단어 간의 유사도를 나타내기 위해 path_similarity() 메서드 제공
- path_similarity()를 이용해 'tree', 'lion', 'tiger', 'cat', 'dog' 단어의 상호 유사도를 살펴보겠음

In [ ]:
# synset 객체를 단어별로 생성
tree = wn.synset('tree.n.01')
lion = wn.synset('lion.n.01')
tiger = wn.synset('tiger.n.02')
cat = wn.synset('cat.n.01')
dog = wn.synset('dog.n.01')

entities = [tree, lion, tiger, cat, dog]
similarities = []
entity_names = [entity.name().split('.')[0] for entity in entities]

# 단어별 synset을 반복하면서 다른 단어의 synset과 유사도를 측정
for entity in entities :
    similarity = [round(entity.path_similarity(compared_entity), 2) for compared_entity in entities]
    similarities.append(similarity)

# 개별 단어별 synset과 다른 단어의 synset과의 유사도를 DF 형태로 저장
similarity_df = pd.DataFrame(similarities, columns=entity_names, index=entity_names)
similarity_df

- lion 과 tree 의 유사도가 0.07로 가장 적음
- lion 과 tiger 의 유사도는 0.33으로 가장 큼




In [ ]:
 import nltk
 from nltk.corpus import sentiwordnet as swn

 senti_synsets = list(swn.senti_synsets('slow'))
 print('senti_synsets() 변환 type:', type(senti_synsets))
 print('senti_synsets() 변환 값 개수:', len(senti_synsets))
 print('senti_synsets() 변환 값:', senti_synsets)

- SentiSynset 객체는 단어의 감성을 나타내는 감성지수와 객관성(감성과 반대)을 나타내는 객관성 지수를 가지고 있음
- 어떤 단어가 전혀 감성적이지 않으면 객관성 지수는 1, 감성 지수는 0이 됨

In [ ]:
import nltk
from nltk.corpus import sentiwordnet as swn

father = swn.senti_synset('father.n.01')

print('father 긍정 감성지수 :', father.pos_score())
print('father 부정 감성지수 :', father.neg_score())
print('father 객관성 지수 :', father.obj_score())
print("\n")

fabulous = swn.senti_synset('fabulous.a.01')
print('fabulous 긍정 감성지수 :', fabulous.pos_score())
print('fabulous 부정 감성지수 :', fabulous.neg_score())
print('fabulous 객관성 지수 :', fabulous.obj_score())

#### SentiWordNet을 이용한 영화 감상평 감성 분석

감성 분석을 수행하는 순서는 다음과 같음
1. 문서(Document)를 문장(Sentence) 단위로 분해
2. 다시 문장을 단어(Word) 단위로 토큰화하고 품사 태깅
3. 품사 태깅된 단어 기반으로 synset 객체와 senti_synset 객체를 생성
4. Senti_synset 에서 긍정 감성 / 부정 감성 지수를 구하고 이를 모두 합산해 특정 임계치 값 이상일 때 긍정 감성으로, 그렇지 않으면 부정으로 결정

- 품사를 태깅하는 내부 함수 생성

In [17]:
from nltk.corpus import wordnet as wn

# 간단한 NLTK PennTreebank Tag를 기반으로 WordNet 기반의 품사 Tag로 변환
def penn_to_wn(tag) :
    if tag.startwith('J') :
        return wn.ADJ
    elif tag.startwith('N') :
        return wn.NOUN
    elif tag.startwith('R') :
        return wn.ADV
    elif tag.startwith('V') :
        return wn.VERB
    

- 문서를 문장 > 단어 토큰 > 품사 태깅 후 Polarity Score를 합산하는 함수 생성

In [18]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag

def swn_polarity(text) :
    # 감성 지수 초기화
    sentiment = 0.0
    tokens_count = 0

    lemmatizer = WordNetLemmatizer()
    raw_sentences = sent_tokenize(text)

    # 분해된 문장별로 단어 토큰 > 품사 태깅 후 SentiSynset 생성 > 감성 지수 합산
    for raw_sentence in raw_sentences :
        # NLTK 기반의 품사 태깅 문장 추출
        tagged_sentence = pos_tag(word_tokenize(raw_sentence))
        for word, tag in tagged_sentence:

            # WordNet 기반 품사 태깅과 어근 추출
            wn_tag = penn_to_wn(tag)
            if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV) :
                continue
            
            lemma = lemmatizer.lemmatize(word, pos=wn_tag)
            if not lemma :
                continue
            
            # 어근을 추출한 단어와 WordNet 기반 품사 태깅을 입력해 Synset 객체를 생성
            synsets = wn.synsets(lemma, pos=wn_tag)
            if not synsets :
                continue

            # sentiwordnet의 감성 단어 분석으로 감성 synset 추출
            # 모든 단어에 대해 긍정 감성 지수는 +로, 부정 감성 지수는 -로 합산해 감성 지수 계산
            synset = synsets[0]
            swn_synset = swn.senti_synset(synset.name())
            sentiment += (swn_synset.pos_score() - swn_synset.neg_score())
            tokens_count += 1
        if not tokens_count :
            return 0

        # 총 score 가 0 이상일 경우 긍정(Positive) 1, 아님 부정(Negative) 0 반환
        if sentiment >= 0 :
            return 1

        return 0

In [ ]:
train_df['preds'] = train_df['review'].apply(lambda x : swn_polarity(x))
y_target = train_df['sentiment'].values
preds = train_df['preds'].values

p. 509